In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso
)
%matplotlib inline

/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
train = pd.read_csv('./input/train.csv') #訓練データ
test = pd.read_csv('./input/test.csv') #テストデータ

train = train.drop("Id", axis = 1)
target = train["SalePrice"]
train = train.drop("SalePrice", axis = 1)


In [4]:
train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [5]:
numerics = train.select_dtypes(include='number').columns.tolist()
numerics

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [6]:
origin_categoricals = train.select_dtypes(exclude='number').columns.tolist()
origin_categoricals
# categoricals[3]

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [7]:
import category_encoders
# categoricals =['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
#        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2']
ce = category_encoders.OrdinalEncoder(cols=origin_categoricals,handle_unknown='impute')
train[origin_categoricals] = ce.fit_transform(train[origin_categoricals])

features = origin_categoricals + numerics


# train["SalePrice"]

In [8]:
low_fis = ['SaleCondition','LowQualFinSF', 'SaleType', 'Heating', 'GarageQual', 'BsmtFinType2', 'GarageCond',
     'RoofMatl', '3SsnPorch', 'PoolArea', 'Condition2', 'Functional', 'PoolQC', 'Utilities', 'Street', 'MiscVal',
           'BldgType', 'MiscFeature', 'RoofStyle', 'Alley',
#          ]

            'MSZoning', 'BsmtHalfBath', 'Foundation', 'Fence', 'KitchenAbvGr', 'HouseStyle', 'GarageType', 
            'MasVnrType', 'Condition1', 'BsmtQual', 'PavedDrive', 'LotConfig', 'LandSlope', 'ExterCond', 
            'Electrical', 'BsmtFinSF2', 'BsmtCond', 'BsmtExposure', 'CentralAir', 'Exterior2nd'
          ]

categoricals = origin_categoricals.copy()

for low_fi in low_fis:
    features.remove(low_fi)
    if low_fi in categoricals:
        categoricals.remove(low_fi)
    
train = train[features]

In [9]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.7,
            'subsample_freq': 1,
            'learning_rate': 0.01,
            'num_leaves': 100,
            'min_data_in_leaf':24,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            #'device':'gpu'
            }

In [10]:

folds = 5
seed = 666

#kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

In [11]:
train_X, val_X , train_y, val_y =  train_test_split(train, target, test_size= 0.2, random_state = 666)


In [12]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)

In [13]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)

/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/yoshimurahiroyuki/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 41151.4	valid_1's rmse: 47627.5
[200]	training's rmse: 27686.2	valid_1's rmse: 36637.3
[300]	training's rmse: 22933	valid_1's rmse: 32602.5
[400]	training's rmse: 20779.4	valid_1's rmse: 30953.6
[500]	training's rmse: 19379.5	valid_1's rmse: 30284.9
[600]	training's rmse: 18248.6	valid_1's rmse: 29744
[700]	training's rmse: 17286.2	valid_1's rmse: 29418.3
[800]	training's rmse: 16400	valid_1's rmse: 29155.7
[900]	training's rmse: 15596	valid_1's rmse: 28949.1
[1000]	training's rmse: 14867.4	valid_1's rmse: 28778.5
[1100]	training's rmse: 14258.2	valid_1's rmse: 28749.9
[1200]	training's rmse: 13669.9	valid_1's rmse: 28683.3
[1300]	training's rmse: 13097.6	valid_1's rmse: 28630.7
[1400]	training's rmse: 12586.1	valid_1's rmse: 28641.7
[1500]	training's rmse: 12119.8	valid_1's rmse: 28630.5
Early stopping, best iteration is:
[1458]	training's rmse: 12314.9	valid_1's rmse: 28575.8


In [14]:
fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
low_fis = fi.sort_values(1).head(20)[0].tolist()
print(low_fis)
# low_fis = ['SaleCondition','LowQualFinSF', 'SaleType', 'Heating', 'GarageQual', 'BsmtFinType2', 'GarageCond',
#      'RoofMatl', '3SsnPorch', 'PoolArea', 'Condition2', 'Functional', 'PoolQC', 'Utilities', 'Street', 'MiscVal', 'BldgType', 'MiscFeature', 'RoofStyle', 'Alley']
# print(categoricals)
# for low_fi in low_fis:
#     print (low_fi)
#     features.remove(low_fi)

['FullBath', 'FireplaceQu', 'HalfBath', 'ScreenPorch', 'BsmtFinType1', 'EnclosedPorch', 'ExterQual', 'BsmtFullBath', 'BedroomAbvGr', 'Exterior1st', 'LotShape', 'Fireplaces', 'GarageCars', 'KitchenQual', 'LandContour', 'GarageFinish', 'HeatingQC', 'MSSubClass', 'TotRmsAbvGrd', 'YrSold']


In [15]:
ce_test = category_encoders.OrdinalEncoder(cols=origin_categoricals,handle_unknown='impute')
test[origin_categoricals] = ce.fit_transform(test[origin_categoricals])

In [16]:

res =gbm.predict(test)



In [17]:
from datetime import datetime
sub = pd.DataFrame()
sub['Id'] = test['Id']
sub['SalePrice'] = res
tmpTime = datetime.now().strftime("%Y%m%d%H%M%S")
filename='submission'+tmpTime+'.csv'
sub.to_csv( filename, index=False)
